In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]
list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

/workspace


DEBUG:__main__:hello
DEBUG:lib.lab_lib:hello


# TODO

1. 相対コスト
    * ✅TAU計測の実行時間による相対コストの算出
    * ✅ベンチマークプログラム実行後に出力される実行時間の取得
    * ベンチマークプログラム実行後に出力される実行時間による相対コストの算出
2. 不要なモデルの削除
3. モデルの採用割合
4. 予測のコストをプロットした奴
5. 誤差率とコストを同じ表に収めた奴


In [2]:
list_series :list[pd.Series] = []
pis :list[str]
for elem_process in [8, 27, 64, 125, 216, 343]+[512, 729, 1000]:
    print (elem_process)

    fileDir :str = "./txt_files/ElapFiles/"
    fileName :str = f"p{elem_process}_Elap"
    with open(fileDir+fileName) as f:
        l :list[str] = [s.strip() for s in f.readlines()]
    if len(l)%2 != 0:
        warnings.warn("ファイルの行数が偶数ではありません")
    for i in range(len(l)):
        if i%2 == 0:
            pis = l[i].split(sep="/")[-3:]
        else:
            time = l[i].split(sep=" ")[-2]
            _series :pd.Series = pd.Series({
                "p":int(pis[0].replace("p", "")),
                "i":int(pis[1].replace("i", "")),
                "s":int(pis[2].replace("s", "")),
                "time":float(time)
            })
            list_series.append(_series)
        # print(f"pis={pis}, time={time}")

DF_pis_time :pd.DataFrame = pd.DataFrame(data=list_series)

DF_pis_time

8
27
64
125
216
343
512
729
1000


,p,i,s,time
0,8.0,8.0,16.0,0.044
1,8.0,8.0,24.0,0.170
2,8.0,8.0,32.0,0.380
3,8.0,8.0,48.0,1.200
4,8.0,8.0,64.0,2.900
...,...,...,...,...
556,1000.0,512.0,96.0,980.000
557,1000.0,512.0,128.0,2300.000
558,1000.0,1024.0,64.0,550.000
559,1000.0,1024.0,96.0,2000.000


In [3]:
target_p :int = 1000
target_i :int = 1024
target_s :int = 128

DF_pis_time[(DF_pis_time["p"]==target_p) & (DF_pis_time["i"]==target_i) & (DF_pis_time["s"]==target_s)]

,p,i,s,time
542,1000.0,1024.0,128.0,4600.0
551,1000.0,1024.0,128.0,4700.0
560,1000.0,1024.0,128.0,4700.0


In [4]:
resVar = "Exclusive"

In [5]:
costToBuildModel :float = 0
for elem_process in train_lulesh_processes:
    for elem_iteration in train_lulesh_iterations:
        for elem_size in train_lulesh_sizes:
            inputDFtoGetCost :pd.DataFrame = ret_averaged_rawDF_lulesh(list_process = [elem_process], list_iteration=[elem_iteration], list_size=[elem_size], list_csvDir=list_csvDir, resVar=resVar)
            costToBuildModel += get_CostAtInputDF(inputDF=inputDFtoGetCost, targetColName=resVar, numOfProcess=elem_process)

print(f"costToBuildModel={costToBuildModel}")


costToBuildModel=79575.00722467963


In [6]:

list_toDFaboutCostAndCondition :list[pd.Series] = []
for elem_process in test_lulesh_processes:
    for elem_iteration in test_lulesh_iterations:
        for elem_size in test_lulesh_sizes:
            inputDFtoGetCost :pd.DataFrame = ret_averaged_rawDF_lulesh(list_process = [elem_process], list_iteration=[elem_iteration], list_size=[elem_size], list_csvDir=list_csvDir, resVar=resVar)
            costInThisCondition :float= get_CostAtInputDF(inputDF=inputDFtoGetCost, targetColName=resVar, numOfProcess=elem_process)
            _series :pd.Series = pd.Series({
                "cost": costInThisCondition,
                "process": elem_process,
                "iteration" : elem_iteration,
                "size": elem_size,
                "relativeCost(=costToBuildModel/cost * 100)" : costToBuildModel/costInThisCondition * 100
            })
            list_toDFaboutCostAndCondition.append(_series)

DFaboutCost :pd.DataFrame = pd.DataFrame(data=list_toDFaboutCostAndCondition)

DFaboutCost

,cost,process,iteration,size,relativeCost(=costToBuildModel/cost * 100)
0,6.766045e+04,512.0,256.0,64.0,117.609342
1,2.416557e+05,512.0,256.0,96.0,32.929089
2,5.856285e+05,512.0,256.0,128.0,13.587966
3,1.348294e+05,512.0,512.0,64.0,59.019038
4,4.893476e+05,512.0,512.0,96.0,16.261448
5,1.175641e+06,512.0,512.0,128.0,6.768646
6,2.744798e+05,512.0,1024.0,64.0,28.991209
7,9.680321e+05,512.0,1024.0,96.0,8.220286
8,2.344354e+06,512.0,1024.0,128.0,3.394325
9,1.042970e+05,729.0,256.0,64.0,76.296510


In [7]:
print(DFaboutCost.to_csv())

,cost,process,iteration,size,relativeCost(=costToBuildModel/cost * 100)
0,67660.44779434668,512.0,256.0,64.0,117.60934167410058
1,241655.66206907728,512.0,256.0,96.0,32.92908866415599
2,585628.5268031145,512.0,256.0,128.0,13.587966361384643
3,134829.38787379197,512.0,512.0,64.0,59.019037673868546
4,489347.61192789325,512.0,512.0,96.0,16.261447953363106
5,1175641.4546334718,512.0,512.0,128.0,6.768645909095527
6,274479.7778389333,512.0,1024.0,64.0,28.99120942577228
7,968032.1129782613,512.0,1024.0,96.0,8.220285893187782
8,2344354.329343829,512.0,1024.0,128.0,3.3943250910775173
9,104297.04679624198,729.0,256.0,64.0,76.29651046605365
10,346765.65203209495,729.0,256.0,96.0,22.947776620423337
11,833179.7265746038,729.0,256.0,128.0,9.550761340753098
12,201359.75723555396,729.0,512.0,64.0,39.518823580816836
13,701192.085256251,729.0,512.0,96.0,11.348531864217906
14,1669530.079918935,729.0,512.0,128.0,4.766311681460897
15,393842.233052451,729.0,1024.0,64.0,20.204792819687782
16,1400878.76330060